# Imports

In [ ]:
import pandas as pd
import numpy as np
import json
import datetime
import ast

from sklearn import preprocessing as pp
import pickle

from sklearn import model_selection as ms
import category_encoders as ce

#from boruta   import BorutaPy
from lightgbm import LGBMClassifier

from sklearn.ensemble        import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
from xgboost                 import XGBRegressor
from sklearn.linear_model    import LogisticRegression, SGDRegressor
from sklearn.neighbors       import KNeighborsRegressor
from sklearn.tree            import DecisionTreeRegressor
from sklearn.ensemble        import RandomForestRegressor, BaggingRegressor
from sklearn.metrics         import accuracy_score, recall_score, precision_score, balanced_accuracy_score, f1_score, roc_curve, confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.naive_bayes     import GaussianNB
from lightgbm                import LGBMRegressor

## Helper Functions

In [ ]:
def test_preparation(X_test, y_test):
    X_test = rescalling(X_test)
    X_test = transform(X_test)
    y_test = log_var_resposta(y_test)
    
    return X_test, y_test


def rescalling (X_test):
    
    mm = pickle.load(open('parameters/mm_avg_delivery_time_days.pkl','rb'))
    X_test['avg_delivery_time_days'] = mm.transform(X_test[['avg_delivery_time_days']])
    
    rs_ar = pickle.load(open('parameters/rs_average_rating.pkl','rb'))
    
    X_test['average_rating'] = rs_ar.transform(X_test[['average_rating']])
    
    rs_nr = pickle.load(open('parameters/rs_number_of_reviews.pkl','rb'))
    
    X_test['number_of_reviews'] = rs_nr.transform(X_test[['number_of_reviews']])
    
    return X_test

def transform (df):
    
    for att in ['brand', 'product', 'seller','Style Code','Pack of', 'Pattern']:
        fe = pickle.load(open(f'parameters/fe_{att}.pkl', 'rb'))
        df = fe.transform(df)
        
    for att in ['Ideal For', 'Reversible', 'Suitable For']:
        te = pickle.load(open(f'parameters/te_{att}.pkl','rb'))
        df = te.transform(df)
    
    return df

def log_var_resposta(df):
    df = np.log1p(df)
    return df

def limpeza (df):
    df = df.drop(columns=['_id', 'pid','description', 'images'])
    # transformando de string para data
    df['crawled_at'] = pd.to_datetime(df['crawled_at']).dt.date

    return df

def feature_engen(df):
    ## out_of_stock - transformar para int 
    df['out_of_stock'] = df['out_of_stock'].astype('int64')

    # brand - substituir por outros
    df['brand'] = df['brand'].apply(lambda x: x.lower())

    # criando feature product
    df['product'] = df[['category', 'sub_category']].apply(lambda x: x['category'] + '_' + x['sub_category'], axis = 1)
    
    import ast

    # pega string de dict e cria uma lista de dicionários
    def f(x):
        try:
            return ast.literal_eval(str(x))   
        except Exception as e:
            print(e)
            return []



    def junta_dict(dict_list):
        dicionario = {}
        for d in dict_list:
            for key, value in d.items():
                dicionario[key] = value
        return dicionario

    list = df['product_details'].apply(lambda x: f(x))

    df_details = pd.DataFrame([junta_dict(row) for row in list], index = list.index)

    df_drop_columns = df_details.drop(columns=['', ' '])

    # Removendo valores faltantes acima de 50%
    limite_nulos = len(df_details) * 0.5  
    df_drop = df_details.dropna(thresh = limite_nulos, axis=1)
    df_new = df_drop.copy()

    # Trocar os NaN por 'outros'

    df_new.loc[df_new['Fabric'].isnull(), 'Fabric'] = 'outros'
    df_new.loc[df_new['Pattern'].isnull(), 'Pattern'] = 'outros'
    df_new.loc[df_new['Style Code'].isnull(), 'Style Code'] = 'outros'
    df_new.loc[df_new['Pack of'].isnull(), 'Pack of'] = 'outros'
    df_new.loc[df_new['Type'].isnull(), 'Type'] = 'outros'
    df_new.loc[df_new['Sleeve'].isnull(), 'Sleeve'] = 'outros'
    df_new.loc[df_new['Fit'].isnull(), 'Fit'] = 'outros'
    df_new.loc[df_new['Ideal For'].isnull(), 'Ideal For'] = 'outros'
    df_new.loc[df_new['Suitable For'].isnull(), 'Suitable For'] = 'outros'
    df_new.loc[df_new['Reversible'].isnull(), 'Reversible'] = 'outros'
    df_new.loc[df_new['Fabric Care'].isnull(), 'Fabric Care'] = 'outros'

    df_new.head()

    def type_sex(value):
        if value == 'Men':
            return 'Men'
        elif value == 'Men, Boys':
            return 'all_ages'
        elif value == 'Boys, Men':
            return 'all_ages'
        elif value == 'Boys, Girls, Men, Women':
            return 'unisex_all_ages'
        elif value == 'Women, Men':
            return 'Unisex'
        else:
            return 'outros'

    df_new['Ideal For'] = df_new['Ideal For'].map(type_sex)

    df = pd.concat([df, df_new], axis=1)
    
    df['cotton'] = df['Fabric'].apply(lambda x: 1 if 'cotton' in x.lower() else 0)
    df['poly'] = df['Fabric'].apply(lambda x: 1 if 'poly' in x.lower() else 0)
    df['not_cotton_or_poly'] = df['Fabric'].apply(lambda x: 1 if ('poly' not in x.lower()) and ('cotton' not in x.lower()) else 0)
    
    return df

# Load Data

In [ ]:
# Opening JSON file
train = open('dataset/raw/train.json')
 
# returns JSON object as 
# a dictionary
data_train_aux = json.load(train)

data_train = pd.json_normalize(data_train_aux, record_path = 'data')
data_train.columns = data_train_aux['columns']
data_train.head()

# Data Description

In [ ]:
# retirar descrição
# retirar imagem
# verificar ids duplicados em id produto
# Verificar outliers no price

## Removendo colunas

In [ ]:
data_train = data_train.drop(columns=['description', 'images'])

## Transformando tipo de dados

In [ ]:
data_train.dtypes

In [ ]:
# transformando de string para data
data_train['crawled_at'] = pd.to_datetime(data_train['crawled_at']).dt.date

## Tratando valores nulos

In [ ]:
data_train.isna().sum()/data_train.shape[0]

In [ ]:
data_train = data_train.dropna(subset=['actual_price'])

In [ ]:
data_train.isna().sum()/data_train.shape[0]

## Entendendo product Details

In [ ]:
import ast

# pega string de dict e cria uma lista de dicionários
def f(x):
    try:
        return ast.literal_eval(str(x))   
    except Exception as e:
        print(e)
        return []

teste = data_train['product_details'].apply(lambda x: f(x))

def junta_dict(dict_list):
    dicionario = {}
    for d in dict_list:
        for key, value in d.items():
            dicionario[key] = value
    return dicionario


df = pd.DataFrame([junta_dict(row) for row in teste])

In [ ]:
df = pd.DataFrame([junta_dict(row) for row in teste])
df.isna().sum().sort_values()

# Feature Engineering

In [ ]:
df4 = data_train.copy()

In [ ]:
## out_of_stock - transformar para int 
df4['out_of_stock'] = df4['out_of_stock'].astype('int64')

# brand - substituir por outros
df4['brand'] = df4['brand'].apply(lambda x: x.lower())

# criando feature product
df4['product'] = df4[['category', 'sub_category']].apply(lambda x: x['category'] + '_' + x['sub_category'], axis = 1)

In [ ]:
import ast

# pega string de dict e cria uma lista de dicionários
def f(x):
    try:
        return ast.literal_eval(str(x))   
    except Exception as e:
        print(e)
        return []



def junta_dict(dict_list):
    dicionario = {}
    for d in dict_list:
        for key, value in d.items():
            dicionario[key] = value
    return dicionario

list = df4['product_details'].apply(lambda x: f(x))

df_details = pd.DataFrame([junta_dict(row) for row in list], index = list.index)

df_drop_columns = df_details.drop(columns=['', ' '])

# Removendo valores faltantes acima de 50%
limite_nulos = len(df_details) * 0.5  
df_drop = df_details.dropna(thresh = limite_nulos, axis=1)
df_new = df_drop.copy()

# Trocar os NaN por 'outros'

df_new.loc[df_new['Fabric'].isnull(), 'Fabric'] = 'outros'
df_new.loc[df_new['Pattern'].isnull(), 'Pattern'] = 'outros'
df_new.loc[df_new['Style Code'].isnull(), 'Style Code'] = 'outros'
df_new.loc[df_new['Pack of'].isnull(), 'Pack of'] = 'outros'
df_new.loc[df_new['Type'].isnull(), 'Type'] = 'outros'
df_new.loc[df_new['Sleeve'].isnull(), 'Sleeve'] = 'outros'
df_new.loc[df_new['Fit'].isnull(), 'Fit'] = 'outros'
df_new.loc[df_new['Ideal For'].isnull(), 'Ideal For'] = 'outros'
df_new.loc[df_new['Suitable For'].isnull(), 'Suitable For'] = 'outros'
df_new.loc[df_new['Reversible'].isnull(), 'Reversible'] = 'outros'
df_new.loc[df_new['Fabric Care'].isnull(), 'Fabric Care'] = 'outros'

df_new.head()

def type_sex(value):
    if value == 'Men':
        return 'Men'
    elif value == 'Men, Boys':
        return 'all_ages'
    elif value == 'Boys, Men':
        return 'all_ages'
    elif value == 'Boys, Girls, Men, Women':
        return 'unisex_all_ages'
    elif value == 'Women, Men':
        return 'Unisex'
    else:
        return 'outros'

df_new['Ideal For'] = df_new['Ideal For'].map(type_sex)

df4 = pd.concat([df4, df_new], axis=1)

## Tratando variaveis respostas nulas
prices = df4[['seller','pid', 'actual_price']].groupby('pid').max().reset_index()

precos_nulos = df4.loc[df4['actual_price'].isna(), ['_id','pid']]

precos_recuperados = pd.merge(precos_nulos, prices, how = 'inner', on = 'pid')

df4_aux = pd.merge(df4, precos_recuperados[['_id', 'actual_price']], on = '_id', how = 'left')
df4['actual_price'] = df4_aux['actual_price_x'].fillna(0) + df4_aux['actual_price_y'].fillna(0)

df4 = df4.dropna(subset=['actual_price'])

# Novas Features de roupa

df4['cotton'] = df4['Fabric'].apply(lambda x: 1 if 'cotton' in x.lower() else 0)
df4['poly'] = df4['Fabric'].apply(lambda x: 1 if 'poly' in x.lower() else 0)
df4['not_cotton_or_poly'] = df4['Fabric'].apply(lambda x: 1 if ('poly' not in x.lower()) and ('cotton' not in x.lower()) else 0)

# EDA

In [ ]:
df5 = df4.copy()

In [ ]:
df5['Pattern'].value_counts().head(10)

# Data Preparation

In [ ]:
df6 = df5.copy()

In [ ]:
df6.isna().sum()

## Split Dataset

In [ ]:
X = df6.drop(['_id','pid','actual_price'], axis=1)
y = df6['actual_price']

X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.2)
data_train = pd.concat([X_train, y_train], axis=1)
data_test = pd.concat([X_test, y_test], axis=1)

## Rescalling

In [ ]:
X_train.columns

### Min Max Scaler

In [ ]:
mm = pp.MinMaxScaler()

X_train['avg_delivery_time_days'] = mm.fit_transform(X_train[['avg_delivery_time_days']])
pickle.dump(mm, open('parameters/mm_avg_delivery_time_days.pkl','wb'))

### Robust Scaler

In [ ]:
rs_average_rating   = pp.RobustScaler()
rs_number_of_reviews = pp.RobustScaler()

X_train['average_rating'] = rs_average_rating.fit_transform(X_train[['average_rating']])
pickle.dump(rs_average_rating, open('parameters/rs_average_rating.pkl','wb'))

X_train['number_of_reviews'] = rs_number_of_reviews.fit_transform(X_train[['number_of_reviews']])
pickle.dump(rs_number_of_reviews, open('parameters/rs_number_of_reviews.pkl','wb'))

## Transformation

### One Hot Enconde

In [ ]:
# 'out_of_stock'

### Target Encode

In [ ]:
for att in ['Ideal For', 'Reversible', 'Suitable For']:
    te = ce.TargetEncoder(cols=[att], handle_unknown = 'return_nan').fit(X_train, y_train)
    pickle.dump(te, open(f'parameters/te_{att}.pkl','wb'))
    X_train = te.transform(X_train)

### Frequency Encode

In [ ]:
for att in ['brand', 'product', 'seller','Style Code','Pack of', 'Pattern']:
    fe =  ce.CountEncoder(cols=[att], normalize=True).fit(X_train)
    pickle.dump(fe,open(f'parameters/fe_{att}.pkl', 'wb'))
    X_train = fe.transform(X_train)

### Response Variable

In [ ]:
y_train = np.log1p(y_train)

## Test Preparation

In [ ]:
X_test_mod, y_test_mod = test_preparation(X_test, y_test)

# Feature Selection

In [ ]:
X_train.columns.tolist()

In [ ]:
'Style Code','Pack of','Ideal For','Suitable For','Reversible',

In [ ]:
features_selected = [
'average_rating',
'number_of_reviews',
'brand',
# 'category',
# 'crawled_at',
'out_of_stock',
'avg_delivery_time_days',
#  'product_details',
'seller',
#  'sub_category',
#  'fabrication_time',
#  'title',
#  'Fabric',
#  'Pattern',
'cotton',
'poly',
'Style Code',
'Pack of',
#  'Type',
#  'Sleeve',
#  'Fit',
'Ideal For',
'Suitable For',
'Reversible',
#  'Fabric Care',
'product'
]

## Importancia das árvores

In [ ]:
# Crie e treine o modelo LGBMClassifier
lgb_model = LGBMRegressor(n_jobs=-1, random_state=42)
X_train_sel = X_train[features_selected]
lgb_model.fit(X_train_sel, y_train)

# Obtenha as importâncias das características do modelo
importances = lgb_model.feature_importances_

# Ordene as características por importância decrescente
indices = np.argsort(importances)[::-1]

# Imprima o ranking das características
print('Feature ranking')
df = pd.DataFrame()

for i, j in zip(X_train_sel.columns, importances):
    aux = pd.DataFrame({'feature': i, 'importance': j}, index=[0])
    df = pd.concat([df, aux], axis=0)

print(df.sort_values('importance', ascending=False))

# Machine Learning

In [ ]:
def MachineLearning(X_train, y_train, X_test, y_test, features_selected):
    SEED = 42

    lista_de_medidas = ['SMAPE']
    nome_das_medidas = ['SMAPE']

    lista_de_modelos = [XGBRegressor(learning_rate=0.1, n_estimators=50, max_depth=3),
                        SGDRegressor(random_state=SEED), 
                        LGBMRegressor(random_state=SEED, n_jobs=-1, force_row_wise=True, ),
                        DecisionTreeRegressor(random_state=SEED),
                        KNeighborsRegressor(n_neighbors=5,  weights='distance',n_jobs=-1),
                        BaggingRegressor(),
                        RandomForestRegressor(random_state=SEED)]

    nome_dos_modelos = ['XGBoost', 
                        'SGD', 
                        'LGBM', 
                        'DecisionTree',
                        'KNN',
                        'Bagging',
                        'RandomForest']

    resultados0 = {}
    
    X_train_selected = X_train[features_selected]
    X_test_selected = X_test[features_selected]


    for i in range(len(lista_de_modelos)):
        print('Rodando modelo: ' + nome_dos_modelos[i])
        
        model = lista_de_modelos[i]
        
        model.fit(X_train_selected, y_train)
        
        y_pred = model.predict(X_test_selected)
        
        smape = 100 / len(y_test) * np.sum(2 * np.abs(np.expm1(y_pred) - np.expm1(y_test)) / (np.abs(np.expm1(y_test)) + np.abs(np.expm1(y_pred))))

        resultados0[nome_dos_modelos[i]] = [smape]
    
    resultados = pd.DataFrame(resultados0, index = nome_das_medidas).T

    return resultados

In [ ]:
MachineLearning(X_train, y_train, X_test_mod, y_test_mod, features_selected)

# Fine Tuning

- Não possui cross validation
- Não realiza reencode das variáveis
- **Para maior assertividade, rodar com todos os dados de treino disponíveis**

In [583]:
# def objective_cv(trial):  
    
#     n_estimators = trial.suggest_int('n_estimators',20, 100)
#     eta = trial.suggest_float('eta',0.01, 0.05)
#     max_depth = trial.suggest_int('max_depth', 3, 9)
#     subsample = trial.suggest_float('subsample',0.1, 0.9)
#     colsample_bytree = trial.suggest_float('colsample_bytree',0.1,0.9)

#     # model definition
#     xgb_model = XGBRegressor( n_estimators    = n_estimators, 
#                               eta              = eta, 
#                               max_depth        = max_depth, 
#                               subsample        = subsample,
#                               colsample_bytree = colsample_bytree
#                              )
    
#     xgb_model.fit(X_train[features_selected], y_train)
    
#     # Model Prediction 
#     y_pred = xgb_model.predict(X_test_mod[features_selected])

#     # calculando rmse
#     smape = 100 / len(y_test_mod) * np.sum(2 * np.abs(np.expm1(y_pred) - np.expm1(y_test_mod)) / (np.abs(np.expm1(y_test_mod)) + np.abs(np.expm1(y_pred))))

#     return smape

In [584]:
# import optuna
# study = optuna.create_study(direction='minimize')
# study.optimize(objective_cv, n_trials=50)

[I 2023-10-22 12:13:02,310] A new study created in memory with name: no-name-b446f611-3929-4a5a-ae79-792fa10aaecf
[I 2023-10-22 12:13:03,632] Trial 0 finished with value: 92.20713033879997 and parameters: {'n_estimators': 99, 'eta': 0.01859040444788452, 'max_depth': 6, 'subsample': 0.790123784953477, 'colsample_bytree': 0.2692136148922196}. Best is trial 0 with value: 92.20713033879997.
[I 2023-10-22 12:13:05,308] Trial 1 finished with value: 51.972385113135196 and parameters: {'n_estimators': 67, 'eta': 0.03878872959436273, 'max_depth': 4, 'subsample': 0.13244512083963694, 'colsample_bytree': 0.8932924532840313}. Best is trial 1 with value: 51.972385113135196.
[I 2023-10-22 12:13:06,190] Trial 2 finished with value: 72.88257113401903 and parameters: {'n_estimators': 43, 'eta': 0.04878155245402561, 'max_depth': 6, 'subsample': 0.2071016730158456, 'colsample_bytree': 0.849612828100148}. Best is trial 1 with value: 51.972385113135196.
[I 2023-10-22 12:13:06,517] Trial 3 finished with val

[I 2023-10-22 12:13:42,402] Trial 29 finished with value: 39.22044992823669 and parameters: {'n_estimators': 100, 'eta': 0.036654248977630366, 'max_depth': 9, 'subsample': 0.8000008171537041, 'colsample_bytree': 0.40510937225478605}. Best is trial 28 with value: 38.31550884111289.
[I 2023-10-22 12:13:43,532] Trial 30 finished with value: 38.46473957427875 and parameters: {'n_estimators': 98, 'eta': 0.04346282777254415, 'max_depth': 7, 'subsample': 0.8844212326536539, 'colsample_bytree': 0.5266865781246111}. Best is trial 28 with value: 38.31550884111289.
[I 2023-10-22 12:13:44,812] Trial 31 finished with value: 38.579194411918884 and parameters: {'n_estimators': 96, 'eta': 0.0438687472701772, 'max_depth': 7, 'subsample': 0.7085867331631717, 'colsample_bytree': 0.5485655087756972}. Best is trial 28 with value: 38.31550884111289.
[I 2023-10-22 12:13:46,032] Trial 32 finished with value: 38.82649263031917 and parameters: {'n_estimators': 96, 'eta': 0.041188436878236694, 'max_depth': 8, 's

In [153]:
# best_params = study.best_params
best_params = {'n_estimators': 100,
 'eta': 0.04847968659990019,
 'max_depth': 7,
 'subsample': 0.31355832178198495,
 'colsample_bytree': 0.5456941887941139}

In [154]:
best_model = XGBRegressor(n_jobs = -1 , 
                                 random_state=0,
                                 n_estimators = best_params['n_estimators'],
                                 eta = best_params['eta'],
                                 max_depth = best_params['max_depth'],
                                 subsample = best_params['subsample'],
                                 colsample_bytree = best_params['colsample_bytree'])

# model fit
best_model.fit(X_train[features_selected], y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.5456941887941139, early_stopping_rounds=None,
             enable_categorical=False, eta=0.04847968659990019,
             eval_metric=None, feature_types=None, gamma=None, gpu_id=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=-1, num_parallel_tree=None,
             predictor=None, ...)

## Submission

### Carregar os dados

In [155]:
# Opening JSON file
sub = open('dataset/raw/test.json')
 
# returns JSON object as 
# a dictionary
data_sub_aux = json.load(sub)

data_sub = pd.json_normalize(data_sub_aux, record_path = 'data')
data_sub.columns = data_sub_aux['columns']
data_sub.shape

(5600, 16)

### Transformar os dados

In [156]:
def submission_preparation(df, features_selected):
    
    df = limpeza(df)
    df = feature_engen(df)
    df = rescalling(df)
    df = transform(df)
    
    return df[features_selected]

### Aplicar sobre os dados de teste

In [157]:
data_sub_mod = submission_preparation(data_sub, features_selected)

In [158]:
y_hat = np.expm1(best_model.predict(data_sub_mod))

In [159]:
df_submission = pd.DataFrame()
df_submission['pid'] = data_sub['pid']
df_submission['actual_price'] = y_hat

In [163]:
df_submission.to_csv('submissions/submission_6.csv', index=False)

------
# 10 Help Functions

In [327]:
def f(x):
    try:
        return ast.literal_eval(str(x))   
    except Exception as e:
        print(e)
        return []

In [328]:
def junta_dict(dict_list):
    dicionario = {}
    for d in dict_list:
        for key, value in d.items():
            dicionario[key] = value
    return dicionario

In [329]:
def feature_engen(df):
    ## out_of_stock - transformar para int 
    df['out_of_stock'] = df['out_of_stock'].astype('int64')

    # brand - substituir por outros
    df['brand'] = df['brand'].apply(lambda x: x.lower())

    # criando feature product
    df['product'] = df[['category', 'sub_category']].apply(lambda x: x['category'] + '_' + x['sub_category'], axis = 1)
    
    ## Tratando variaveis respostas nulas
    prices = df[['seller','pid', 'actual_price']].groupby('pid').max().reset_index()
    precos_nulos = df.loc[df['actual_price'].isna(), ['_id','pid']]
    precos_recuperados = pd.merge(precos_nulos, prices, how = 'inner', on = 'pid')
    
    df_aux = pd.merge(df, precos_recuperados[['_id', 'actual_price']], on = '_id', how = 'left')
    df['actual_price'] = df_aux['actual_price_x'].fillna(0) + df_aux['actual_price_y'].fillna(0)
    
    df = df.dropna(subset=['actual_price'])
    
    return df

In [330]:
def pipeline (df):
    df = limpeza(df)
    df = feature_engen(df)
    
    product_details = df['product_details'].apply(lambda x: f(x))
    product_details = pd.DataFrame([junta_dict(row) for row in product_details])
    
    return df, product_details


In [331]:
def test_preparation(df):
    X_test = rescalling(X_test)
    X_test = transform(X_test)
    y_test = log_var_resposta(y_test)
    
    return X_test

In [332]:
# '_id'
# 'average_rating':         muitos outliers mas distribuição proxima da normal
#                           tratar notas que não possuem avaliação
# 'number_of_reviews':      muitos outliers mas distribuição proxima da normal
#                           ver como fica a distribuição como log
# 'brand':                  12 categorias frequency -> transformar tudo em minusculo para evitar duplicações
# 'category':               Uma categoria é mais majoritária 
#                           unir com a subcategory fazendo o tratamento de maiusculas
# 'crawled_at':             Somente 1 dia, muda somente a hora -> retirar XXXXX
# 'out_of_stock':           Dummie enconde (somente 0 ou 1) -> One hot encode
# 'avg_delivery_time_days': sem outliers e distribuição normal -> min max scaler
# 'pid':                    valore duplicatidos, muitas labels -> usar para preencher valores nulos no price mas não vai para o modelo
# 'seller':                 Reunir undefined em "outros" e usar o frequency encode
# 'sub_category':           Unir com o 'category'
# 'fabrication_time':       sem outliers, distribuição uniforme -> min max scaler
# 'title':                  muitas categorias, sem concentrações -> frequency 
#                           explorar mais o title
#                           deletar nesse momento
# 'actual_price':           transformar em escala log